In [1]:
import tensorflow as tf

In [2]:
deep_learning = tf.constant('Deep Learning')
session=tf.Session()
session.run(deep_learning)


'Deep Learning'

In [3]:
a=tf.constant(2)
b=tf.constant(2)
multiply=tf.multiply(a,b)
session.run(multiply)

4

In [4]:
weights = tf.Variable(tf.random_normal([10,10],stddev=0.5),name='weights')
weights

<tf.Variable 'weights:0' shape=(10, 10) dtype=float32_ref>

In [7]:
from read_data import get_minibatch
x = tf.placeholder(tf.float32,name="x",shape=[None,784])
W = tf.Variable(tf.random_uniform([784,10],-1,1),name='W')
b = tf.Variable(tf.zeros([10]),name='biases')
output = tf.matmul(x,W)+b

init_op = tf.initialize_all_variables()

sess=tf.Session()
sess.run(init_op)
feed_dict = {"x":get_minibatch()}
sess.run(output,feed_dict=feed_dict)

ImportError: No module named read_data

### MNIST logistic regression

In [1]:
import tensorflow as tf
def inference(x):
    """produce a probability distribution over output classes"""
    tf.constant_initializer(value=0)
    W=tf.get_variable("W",[784,10])#,initializer=init)
    b=tf.get_variable('b',[10])#,initializer=init)
    output=tf.nn.softmax(tf.matmul(x,W)+b)
    return output
def loss(output,y):
    """compute the value of the error function"""
    dot_prod = y * tf.log(output)
    xentropy=-tf.reduce_sum(dot_prod,reduction_indices=1)
    
    # Reduction along axis 0 collapses each column into a
    # single value, whereas reduction along axis 1 collapses
    # each row into a single value. In general, reduction along
    # axis i collapses the ith dimension of a tensor to size 1.
    xentropy = -tf.reduce_sum(dot_prod, reduction_indices=1)
    
    loss = tf.reduce_mean(xentropy)
    return loss
def training(cost,global_step):
    """compute the gradients of the""" 
    """model's parameters and updating the model"""
    # scalar summary statistic for the cost function
    #log the cost of each minibatch,validation error, 
    #and the distribution of parameters
    tf.summary.scalar("cost", cost)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost,global_step=global_step)
    return train_op
def evaluate(output,y):
    """determine the effectiveness of a model"""
    correct_prediction = tf.equal(tf.argmax(output,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    return accuracy

In [3]:
# sudo pip install python-mnist
import mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
tf.reset_default_graph()
##### PARAMETERS ####
learning_rate= 0.01
training_epochs=1000
batch_size=100
display_step=100
#####################
with tf.Graph().as_default():
    # mnist data image shape 28*28=784
    x=tf.placeholder("float",[None,784])
    # 0-9 digits recognition >=10 classes
    y=tf.placeholder("float",[None,10])
    
    output=inference(x)
    cost = loss(output,y)
    global_step= tf.Variable(0,name='global_step',trainable=False)
    train_op = training(cost,global_step)
    eval_op = evaluate(output,y)
    summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    sess=tf.Session()
    summary_writer=tf.summary.FileWriter("logistic_logs/",graph_def=sess.graph_def)
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    #### TRAINING CYCLE ####
    for epoch in range(training_epochs):
        avg_cost=0.
        total_batch=int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            mbatch_x,mbatch_y= mnist.train.next_batch(batch_size)
            # Fit training with batch
            feed_dict = {x:mbatch_x,y:mbatch_y}
            sess.run(train_op,feed_dict=feed_dict)
            # Compute average loss
            minibatch_cost = sess.run(cost,feed_dict=feed_dict)
            avg_cost += minibatch_cost/total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
                val_feed_dict = {
                            x: mnist.validation.images,
                            y: mnist.validation.labels
                }
                accuracy = sess.run(eval_op,feed_dict=val_feed_dict)
                print "Validation Error:", (1-accuracy)
                summary_str = sess.run(summary_op,feed_dict=feed_dict)
                summary_writer.add_summary(summary_str,sess.run(global_step))
                saver.save(sess,"logistic_logs/model-checkpoint",global_step=global_step)
                summary_writer.flush()
    print "---OPTIMIZATION COMPLETE---"
    test_feed_dict= {
        x:mnist.test.images,
        y:mnist.test.labels
    }
    accuracy = sess.run(eval_op,feed_dict=test_feed_dict)
    print "Test Accuracy: {}".format(accuracy)
    ########################

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Validation Error: 0.168600022793
Validation Error: 0.0759999752045
Validation Error: 0.0738000273705
Validation Error: 0.071799993515
Validation Error: 0.0708000063896
Validation Error: 0.0702000260353
Validation Error: 0.0702000260353
Validation Error: 0.0695999860764
Validation Error: 0.0694000124931
Validation Error: 0.0699999928474
---OPTIMIZATION COMPLETE---
Test Accuracy: 0.926299989223
